In [1]:
import pandas as pd
import numpy as np
import cx_Oracle
import pymongo
from py2neo import Graph, Node, Relationship, NodeMatcher
import warnings

# Desabilitar o aviso temporariamente
warnings.filterwarnings("ignore", category=FutureWarning)

# Conexões

In [2]:
instant_client = "./instantclient_21_9"
cx_Oracle.init_oracle_client(lib_dir=instant_client)
# Conectar ao banco de dados Oracle
conn = cx_Oracle.connect(user="store", password="pass", dsn="localhost:1521/xe")

# estabelecer conexão com o servidor MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017")

# estabelecer conexão neo4j
graph = Graph("bolt://localhost:7687 ", auth=("neo4j", "password"))
matcher = NodeMatcher(graph)

# SQL

In [3]:
tabelasSQL = ["STORE_USERS", "PRODUCT_CATEGORIES", "PRODUCT", "DISCOUNT", "CART_ITEM", "SHOPPING_SESSION", "ORDER_DETAILS", "ORDER_ITEMS", "PAYMENT_DETAILS", "EMPLOYEES", "DEPARTMENTS", "ADRESSES", "EMPLOYEES_ARCHIVE", "STOCK"]

# Product
product_categories_fields = ["category_id", "category_name"]
product_fields = ["product_id", "product_name", "category_id", "sku", "price", "discount_id", "created_at", "last_modified"]
discount_fields = ["discount_id", "discount_name", "discount_desc", "discount_percent", "is_active_status", "created_at", "modified_at"]
stock_fields = ["product_id", "quantity", "max_stock_quantity", "unit"]

# Employees
employees_fields = ["employee_id", "first_name", "middle_name", "last_name", "date_of_birth", "department_id", "hire_date", "salary", "phone_number", "email", "ssn_number", "manager_id"]
departments_fields = ["department_id", "department_name", "manager_department_id", "department_desc"]

# Empolyees Archive
employees_archive_fields = ["event_date", "event_type", "user_name", "old_employee_id", "old_first_name", "old_middle_name", "old_last_name", "old_date_of_birth", "old_department_id", "old_hire_date", "old_salary", "old_phone_number", "old_email", "old_ssn_number", "old_manager_id", "new_employee_id", "new_first_name", "new_middle_name", "new_last_name", "new_date_of_birth", "new_department_id", "new_hire_date", "new_salary", "new_phone_number", "new_email", "new_ssn_number", "new_manager_id"]

# ORDER_DETAILS
order_details_fields = ["order_details_id", "user_id", "total", "payment_id", "shipping_method", "delivery_adress_id", "created_at", "modified_at"]
#                          payment_details_id
payment_details_fields = ["payment_id", "order_details_id", "amount", "provider", "payment_status", "payment_created_at", "payment_modified_at"]

# ADDRESES
addresses_fields = ["adress_id", "line_1", "line_2", "city", "zip_code", "province", "country"]

# STORE_USERS
store_users_fields = ["user_id", "first_name", "middle_name", "last_name", "phone_number", "email", "username", "user_password", "registered_at"]

# CART_ITEM
cart_item_fields = ["cart_item_id", "session_id", "product_id", "quantity", "created_at", "modified_at"]

# SHOPPING_SESSION
shopping_session_fields = ["session_id", "user_id", "created_at", "modified_at"]

# ORDER_ITEMS
order_items_fields = ["order_items_id", "order_details_id", "product_id", "created_at", "modified_at"]

In [4]:
# Criar um cursor
cursor = conn.cursor()

### PRODUCT

#### PRODUCT

In [5]:
cursor.execute('SELECT * FROM PRODUCT')
product = cursor.fetchall()

product_df = pd.DataFrame(product)
product_df.columns = product_fields

#### PRODUCT CATEGORIES

In [6]:
cursor.execute('SELECT * FROM PRODUCT_CATEGORIES')
product_categories = cursor.fetchall()

product_categories_df = pd.DataFrame(product_categories)
product_categories_df.columns = product_categories_fields

In [7]:
productMongo_df = pd.merge(product_df, product_categories_df, on='category_id')
productMongo_df = productMongo_df.loc[:, ~productMongo_df.columns.duplicated()]

#### DISCOUNT

In [8]:
cursor.execute('SELECT * FROM DISCOUNT')
discount = cursor.fetchall()

discount_df = pd.DataFrame(discount)
discount_df.columns = discount_fields

nanLinha = pd.Series(np.nan, index=discount_fields)
discount_df = discount_df.append(nanLinha, ignore_index=True)

In [9]:
productMongo_df = pd.merge(productMongo_df, discount_df, on='discount_id')
productMongo_df = productMongo_df.loc[:, ~productMongo_df.columns.duplicated()]
productMongo_df = productMongo_df.replace({None: np.nan})

In [10]:
productMongo_df.shape

(20, 15)

#### STOCK

In [11]:
cursor.execute('SELECT * FROM STOCK')
stock = cursor.fetchall()

stock_df = pd.DataFrame(stock)
stock_df.columns = stock_fields

In [12]:
productMongo_df = pd.merge(productMongo_df, stock_df, on='product_id')

### EMPLOYEES

#### EMPLOYEES

In [13]:
cursor.execute('SELECT * FROM EMPLOYEES')
employees = cursor.fetchall()

employees_df = pd.DataFrame(employees)
employees_df.columns = employees_fields

In [14]:
employees_df.shape

(10, 12)

#### DEPARTMENT

In [15]:
cursor.execute('SELECT * FROM DEPARTMENTS')
departments = cursor.fetchall()

departments_df = pd.DataFrame(departments)
departments_df.columns = departments_fields

In [16]:
employees_mongo_df = pd.merge(employees_df, departments_df, on='department_id')
employees_mongo_df = employees_mongo_df.loc[:, ~employees_mongo_df.columns.duplicated()]

### EMPLOYEES_ARCHIVE

In [17]:
cursor.execute('SELECT * FROM EMPLOYEES_ARCHIVE')
employees_archive = cursor.fetchall()

employees_archive_df = pd.DataFrame(employees_archive)
employees_archive_df.columns = employees_archive_fields

In [18]:
employees_archive_df.shape

(10, 27)

### STORE_USERS

In [19]:
cursor.execute('SELECT * FROM STORE_USERS')
store_users = cursor.fetchall()

store_users_df = pd.DataFrame(store_users)
store_users_df.columns = store_users_fields

### SHOPPING_SESSION

In [20]:
cursor.execute('SELECT * FROM SHOPPING_SESSION')
shopping_session = cursor.fetchall()

shopping_session_df = pd.DataFrame(shopping_session)
shopping_session_df.columns = shopping_session_fields

### CART_ITEM

In [21]:
cursor.execute('SELECT * FROM CART_ITEM')
cart_item = cursor.fetchall()

cart_item_df = pd.DataFrame(cart_item)
cart_item_df.columns = cart_item_fields

In [22]:
cart_item_df.shape

(33, 6)

### ORDER_ITEMS

In [23]:
cursor.execute('SELECT * FROM ORDER_ITEMS')
order_items = cursor.fetchall()

order_items_df = pd.DataFrame(order_items)
order_items_df.columns = order_items_fields

### ORDER_DETAILS

In [24]:
cursor.execute('SELECT * FROM ORDER_DETAILS')
order_details = cursor.fetchall()

order_details_df = pd.DataFrame(order_details)
order_details_df.columns = order_details_fields

#### PAYMENT_DETAILS

In [25]:
cursor.execute('SELECT * FROM PAYMENT_DETAILS')
payment_details = cursor.fetchall()

payment_details_df = pd.DataFrame(payment_details)
payment_details_df.columns = payment_details_fields

In [26]:
order_details_mongo_df = pd.merge(order_details_df, payment_details_df, on='payment_id')
order_details_mongo_df = order_details_mongo_df.loc[:, ~order_details_mongo_df.columns.duplicated()]
order_details_mongo_df.drop("order_details_id_y", axis=1, inplace=True)
order_details_mongo_df.rename(columns={"order_details_id_x": "order_details_id"}, inplace=True) 

### ADDRESSES

In [27]:
cursor.execute('SELECT * FROM ADDRESSES')
addresses = cursor.fetchall()

addresses_df = pd.DataFrame(addresses)
addresses_df.columns = addresses_fields

# Inserir MongoDB

In [28]:
mongo_database = client["Electronics_Store"]
collection_product = mongo_database["product"] 
collection_employees = mongo_database["employees"] 
collection_employees_archive = mongo_database["employees_archive"] 
collection_order_details = mongo_database["order_details"]
collection_store_users = mongo_database["store_users"] 
collection_cart_item = mongo_database["cart_item"] 
collection_shopping_session = mongo_database["shopping_session"] 
collection_order_items = mongo_database["order_items"] 
collection_addresses = mongo_database["addresses"]

### PRODUCT

In [29]:
productMongo_df.replace({pd.NaT: None}, inplace=True)

product_data = productMongo_df.to_dict(orient='records')
collection_product.insert_many(product_data)

### EMPLOYEES

In [30]:
employees_data = employees_mongo_df.to_dict(orient='records')
collection_employees.insert_many(employees_data)

### Empolyees Archive

In [31]:
employees_archive_data = employees_archive_df.to_dict(orient='records')
collection_employees_archive.insert_many(employees_archive_data)

### STORE_USERS

In [32]:
store_users_data = store_users_df.to_dict(orient='records')
collection_store_users.insert_many(store_users_data)

### SHOPPING_SESSION

In [33]:
shopping_session_data = shopping_session_df.to_dict(orient='records')
collection_shopping_session.insert_many(shopping_session_data)

### CART_ITEM

In [34]:
cart_item_df.replace({pd.NaT: None}, inplace=True)

cart_item_data = cart_item_df.to_dict(orient='records')
collection_cart_item.insert_many(cart_item_data)

### ORDER_ITEMS

In [35]:
order_items_df.replace({pd.NaT: None}, inplace=True)

order_items_data = order_items_df.to_dict(orient='records')
collection_order_items.insert_many(order_items_data)

### ORDER_DETAILS

In [36]:
order_details_mongo_df.replace({pd.NaT: None}, inplace=True)

order_details_data = order_details_mongo_df.to_dict(orient='records')
collection_order_details.insert_many(order_details_data)

### ADRESSES

In [37]:
addresses_data = addresses_df.to_dict(orient='records')
collection_addresses.insert_many(addresses_data)